In [1]:
import sympy as sp 
from sympycalcs import render, convert
import sympy.physics.units as unit
from sympy.abc import *

import matplotlib.pyplot as plt

sp.init_printing(use_latex='mathjax', latex_mode='equation')

## Beispiel: Eigenvektoren mit Nachgiebigkeitsmatrix

Das System in @fig-system zeigt einen Rahmen, welcher als Zweimassenschwinger modelliert werden kann.


![Statisches System des 2-Massenschwingers](bilder/mms1.jpg){#fig-system width=50%}

Gesucht:

-  Eigenkreisfrequenz $\omega$ 
-  Eigenformen - Normierung auf 
$$\phi_1^T = 
\begin{bmatrix}
1 &  \\
\end{bmatrix} $$
$$\phi_2^T =
\begin{bmatrix}
1 &  \\
\end{bmatrix}$$

- Skizze der Eigenformen 

Gegeben:

- Biegesteifigkeit aller Stäbe $E\cdot I = 20\cdot 10^{12} \text{Nmm}^2$
- Dehnsteifigkeit aller Stäbe $E\cdot A = \infty$
- Einzelmassen $m_1 = 1000 \text{kg}$ und $m_2 = 1000 \text{kg}$
- Länge $l = 4000 \text{mm}$

{{< pagebreak >}}
### Musterlösung

In [2]:
params = {'EI': 20*10**12 *unit.N*unit.mm**2,
          'm_1':1000*unit.N*unit.second**2/unit.m,
          'm_2':1000*unit.N*unit.second**2/unit.m,
          'l':4*unit.m,
          }

          
render.dict_to_table(params)

| Parameter | - |
|---|---|
| $EI = 20000000000000 \text{mm}^{2} \text{N}$ | $m_{1} = \frac{1000 \text{N} \text{s}^{2}}{\text{m}}$ |
| $m_{2} = \frac{1000 \text{N} \text{s}^{2}}{\text{m}}$ | $l = 4 \text{m}$ |


In [3]:
omega, t,  l, m_1, m_2, EI = sp.symbols('omega, t,  l, m_1, m_2, EI')

omega_n = sp.symbols('omega_n')

#### Nachgiebigkeitsmatrix $\mathbf{D}$ {#sec-nachgiebigkeit}

Die Steifigkeitsmatrix lässt sich durch Invertierung der Nachgiebigkeitsmatrix beschreiben. Die Nachgiebigkeitsmatrix $\mathbf{D}$ beschreibt die Deformation an einem Massenpunkt. Die Einträge der $\mathbf{D}$ - Matrix beschreiben die Deformationen für unterschiedliche Laststellungen.

$$\mathbf{K} = \mathbf{D^{-1}}$$

![Balken mit 2 Einzelmassen](bilder/bsp_nachgib.jpg){#fig-2mms}

Für einen 2-Massenschwinger, wie in @fig-2mms , hat die Nachgiebigkeitsmatrix folgende Form:

$$\mathbf{D} = \frac{1}{EI}\cdot \begin{bmatrix}
\delta_{11} & \delta_{12}\\
\delta_{21} & \delta_{22} 
\end{bmatrix}$$

wobei gilt:

$\delta_{ab}$ : $a$ ist die Lastsituation, $b$ ist die Masse.


##### Anwendung 

![Schnittgrössen für beide Lastfälle zur Bestimmung der Deformation](bilder/rahmen_def.jpg){#fig-schnittgrössen}

$$\delta_{ab} = \frac{1}{EI}\int_{0}^{l} M_a\bar{M_b} \,dx$$

Es werden 2 Laststellungen betrachtet, jeweils an einem Massenpunkt. Dabei ist Beachtung der Einheit der Einwirkung zu schenken. Diese wird einheitslos angesetzt.

In [4]:
delta_11 = 1/(EI) * (sp.Rational(1,3) * -l*-l*1*l)* 2
delta_12 = 1/(EI) * ((sp.Rational(1,3) * -l*-l*1*l) + (sp.Rational(1,2) * -l*-l*1*l))
delta_21 = delta_12
delta_22 = 1/(EI) * ((sp.Rational(1,3) * -l*-l*1*l)* 2 + -l*-l*1*l)

render.eq_display('delta_11', delta_11,
                  'delta_12', delta_12,
                  'delta_21', delta_21,
                  'delta_22', delta_22)


         3
      2⋅l 
δ₁₁ = ────
      3⋅EI

         3
      5⋅l 
δ₁₂ = ────
      6⋅EI

         3
      5⋅l 
δ₂₁ = ────
      6⋅EI

         3
      5⋅l 
δ₂₂ = ────
      3⋅EI

In [5]:
D = sp.Matrix([[delta_11, delta_12],[delta_21, delta_22]])
K = D.inv()
render.eq_display(sp.MatrixSymbol('D', 2,2), D,
                  sp.MatrixSymbol('K', 2,2), K)

    ⎡   3     3⎤
    ⎢2⋅l   5⋅l ⎥
    ⎢────  ────⎥
    ⎢3⋅EI  6⋅EI⎥
D = ⎢          ⎥
    ⎢   3     3⎥
    ⎢5⋅l   5⋅l ⎥
    ⎢────  ────⎥
    ⎣6⋅EI  3⋅EI⎦

    ⎡ 4⋅EI   -2⋅EI ⎤
    ⎢ ────   ──────⎥
    ⎢   3       3  ⎥
    ⎢  l       l   ⎥
K = ⎢              ⎥
    ⎢-2⋅EI    8⋅EI ⎥
    ⎢──────   ──── ⎥
    ⎢   3        3 ⎥
    ⎣  l      5⋅l  ⎦

#### Eigenvektoren

Die Bewegungsgleichung für einen ungedämpften, frei schwingenden Mehrmassenschwinger lässt sich folgender massen aufstellen:

$$\mathbf{M u''(t) + K u(t)} = 0$$

Die Modale Analyse entkoppelt die Gleichungen um diese unabhängig von einander zu lösen.

##### Massenmatrix $\mathbf{M}$

In [6]:
M = sp.Matrix([[m_1,0],[0,m_2]])

render.eq_display(sp.MatrixSymbol('M',2,2), M)

    ⎡m₁  0 ⎤
M = ⎢      ⎥
    ⎣0   m₂⎦

##### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand @eq-eigenkreisfrequenzen bestimmen:

$$
\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}
$${#eq-eigenkreisfrequenzen}

In [7]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

      12.1 
ω₁ = ──────
     second

      40.0 
ω₂ = ──────
     second

##### Eigenvektoren $\mathbf{\phi}$
$$
\phi_n = \begin{bmatrix}
\phi_{1n}\\
\phi_{2n} 
\end{bmatrix}
$$
$$
[\mathbf{K}-\omega_n^2 \mathbf{M}]\cdot \begin{bmatrix}
\phi_{1n}\\
\phi_{2n} 
\end{bmatrix}
=0$$



Dazu ist die entsprechende Normierung aus der Aufgabenstellung zu berücksichtigen.

In [8]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')
params['phi_11'] = 1

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_21 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_21).values())[0]

params['phi_21'] = phi_21.subs(params).simplify() 




render.eq_display((K-omega_1**2 *M)*phi_1,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))


⎡                 ⎛          ⎛                          ______________________
⎢                 ⎜          ⎜                         ╱     2                
⎢  2⋅EI⋅φ₂₁       ⎜4⋅EI      ⎜  4⋅EI     2⋅EI   2⋅EI⋅╲╱  4⋅m₁  + 5⋅m₁⋅m₂ + 25⋅
⎢- ──────── + φ₁₁⋅⎜──── - m₁⋅⎜─────── + ───── - ──────────────────────────────
⎢      3          ⎜  3       ⎜   3       3                     3              
⎢     l           ⎝ l        ⎝5⋅l ⋅m₂   l ⋅m₁               5⋅l ⋅m₁⋅m₂        
⎢                                                                             
⎢                 ⎛          ⎛                          ______________________
⎢                 ⎜          ⎜                         ╱     2                
⎢  2⋅EI⋅φ₁₁       ⎜8⋅EI      ⎜  4⋅EI     2⋅EI   2⋅EI⋅╲╱  4⋅m₁  + 5⋅m₁⋅m₂ + 25⋅
⎢- ──────── + φ₂₁⋅⎜──── - m₂⋅⎜─────── + ───── - ──────────────────────────────
⎢      3          ⎜   3      ⎜   3       3                     3              
⎣     l           ⎝5⋅l       ⎝5⋅l ⋅m₂   l ⋅m₁       

     ⎡1.0 ⎤
φ₁ = ⎢    ⎥
     ⎣1.77⎦

In [9]:
params['phi_12'] = 1

phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_22 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_22).values())[0]

params['phi_22'] = phi_22.subs(params).simplify() 




render.eq_display((K-omega_2**2 *M)*phi_2,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))


⎡                 ⎛          ⎛                          ______________________
⎢                 ⎜          ⎜                         ╱     2                
⎢  2⋅EI⋅φ₂₂       ⎜4⋅EI      ⎜  4⋅EI     2⋅EI   2⋅EI⋅╲╱  4⋅m₁  + 5⋅m₁⋅m₂ + 25⋅
⎢- ──────── + φ₁₂⋅⎜──── - m₁⋅⎜─────── + ───── + ──────────────────────────────
⎢      3          ⎜  3       ⎜   3       3                     3              
⎢     l           ⎝ l        ⎝5⋅l ⋅m₂   l ⋅m₁               5⋅l ⋅m₁⋅m₂        
⎢                                                                             
⎢                 ⎛          ⎛                          ______________________
⎢                 ⎜          ⎜                         ╱     2                
⎢  2⋅EI⋅φ₁₂       ⎜8⋅EI      ⎜  4⋅EI     2⋅EI   2⋅EI⋅╲╱  4⋅m₁  + 5⋅m₁⋅m₂ + 25⋅
⎢- ──────── + φ₂₂⋅⎜──── - m₂⋅⎜─────── + ───── + ──────────────────────────────
⎢      3          ⎜   3      ⎜   3       3                     3              
⎣     l           ⎝5⋅l       ⎝5⋅l ⋅m₂   l ⋅m₁       

     ⎡ 1.0  ⎤
φ₂ = ⎢      ⎥
     ⎣-0.566⎦

##### Orthogonalitätsbedingung
Zur effektiven Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein. Durch die Orthogonalität der Vektoren $\phi_1$ und $\phi_2$ kann mittels einem Einmassenschwingers sämtliches Verhalten von $\phi_1$ beschrieben werden und mittels einem zweiten Einmassenschwinger sämtliches Verhalten von $\phi_2$. Ist die Orthogonalität nicht gegeben, so müsste der erste Einmassenschwinger Anteile aus $\phi_1$ und $\phi_2$ beschreiben.

Dies gilt es für die Massenmatrix zu kontrollieren:

$$\phi_1^T \mathbf{ M } \phi_1 \neq 0$$

$$\phi_2^T \mathbf{ M } \phi_2 \neq 0$$

$$\phi_2^T \mathbf{ M } \phi_1 = 0$$

Sowohl auch für die Steifigkeitsmatrix:

$$\phi_1^T \mathbf{ K } \phi_1 \neq 0$$

$$\phi_2^T \mathbf{ K } \phi_2 \neq 0$$

$$\phi_2^T \mathbf{ K } \phi_1 = 0$$


Angewendet auf die Aufgabe:

In [10]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_2.T*M*phi_1).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(3))

           ⎡                     2⎤
  T        ⎢4.12e+3⋅newton⋅second ⎥
φ₁ ⋅M⋅φ₁ = ⎢──────────────────────⎥
           ⎣        meter         ⎦

           ⎡                     2⎤
  T        ⎢1.32e+3⋅newton⋅second ⎥
φ₂ ⋅M⋅φ₂ = ⎢──────────────────────⎥
           ⎣        meter         ⎦

  T           
φ₂ ⋅M⋅φ₁ = [0]

  T           
φ₁ ⋅M⋅φ₂ = [0]

Für die Steifigkeitsmatrix:

In [11]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(3))

  T        ⎡6.02e+5⋅newton⎤
φ₁ ⋅K⋅φ₁ = ⎢──────────────⎥
           ⎣    meter     ⎦

  T        ⎡2.12e+6⋅newton⎤
φ₂ ⋅K⋅φ₂ = ⎢──────────────⎥
           ⎣    meter     ⎦

  T           
φ₂ ⋅K⋅φ₁ = [0]

  T           
φ₁ ⋅K⋅φ₂ = [0]

#### Eigenformen

![Die beiden Eigenformen skizziert](bilder/eigenformen.jpg){#fig-eigenformen}

{{< pagebreak >}}

## Beispiel: Eigenvektoren mit direkt bestimmter Steifigkeitsmatrix

Das System in @fig-system_mms2 zeigt ein Rahmentragwerk, welches als Zweimassenschwinger modelliert werden kann.

![Statisches System des Rahmentragwerks](bilder/system_mms2.jpg){#fig-system_mms2}

Gesucht:

-  Eigenkreisfrequenz $\omega$ 
-  Eigenformen - Normierung auf 
$$\phi_1^T = 
\begin{bmatrix}
 &  1\\
\end{bmatrix} $$
$$\phi_2^T =
\begin{bmatrix}
 &  1\\
\end{bmatrix}$$

- Skizze der Eigenformen 

Gegeben:

- Elastizitätsmodul der Stützen $E = 30000 \text{N}/{mm}^2$
- Flächenträgheitsmoment der Stützen $I = 2 \cdot 10^9 \text{mm}^4$
- Dehnsteifigkeit aller Stäbe $E\cdot A = \infty$
- Einzelmassen $m = 20000 \text{kg}$

{{< pagebreak >}}

### Musterlösung

In [12]:
params = {'E': 30*10**3 *unit.N/unit.mm**2,
          'I':2*10**9*unit.mm**4,
          'm_1':2*20000*unit.N*unit.second**2/unit.m,
          'm_2':20000*unit.N*unit.second**2/unit.m,
          'l':4*unit.m,
          'H':3.2*unit.m,
          }


render.dict_to_table(params)

| Parameter | - |
|---|---|
| $E = \frac{30000 \text{N}}{\text{mm}^{2}}$ | $I = 2000000000 \text{mm}^{4}$ |
| $m_{1} = \frac{40000 \text{N} \text{s}^{2}}{\text{m}}$ | $m_{2} = \frac{20000 \text{N} \text{s}^{2}}{\text{m}}$ |
| $l = 4 \text{m}$ | $H = 3.2 \text{m}$ |


In [13]:
omega, t,  l, m_1, m_2, E,I, H = sp.symbols('omega, t,  l, m_1, m_2, E, I, H')

omega_n = sp.symbols('omega_n')

#### Eigenkreisfrequenzen
##### Steifigkeitsmatrix $\mathbf{K}$

Zur Bestimmung der Steifigkeitsmatrix ist das System an jedem Freiheitsgrad auszulenken, wie in @fig-steifigkeit dargestellt ist.

![Auslenkung der Freiheitsgrade zur Bestimmung der Steifigkeit](bilder/steifigkeit_mms2.jpg){#fig-steifigkeit}

Wichtig dabei sind die Richtungen der Kräfte. Als Denkstütze gilt folgendes:

- Der Auslenkung um $u$ wirkt die Federkraft entgegen, welche $k u$ entspricht.
- Zusätzlich wirkt die Trägheitskraft der Auslenkung entgegen, welche $m u''$ entspricht.
- Nach der Betrachtung des ausgelenkten Punkts, kann mittels *Actio-Reactio*-Prinzip das "*Stockwerk*" ins Gleichgewicht gebracht werden.
- Vorzeichen sind gegen der Bewegungsrichtig positiv.

##### Horizontale Steifigkeit

Für entsprechende Anwendungsfälle gibt es fertige Lösungen zur Bestimmung der Steifigkeit. Gemäss @fig-system_mms2 ist die Stütze am Fuss- und Kopfpunkt eingespannt. Somit resultiert die Steifigkeit zu:

$$k_{Stuetze} = \frac{12EI_{Stuetze}}{H^3}$$

Diese gilt für eine einzelne Stütze.

In [14]:
k_1, k_2 = sp.symbols('k_1, k_2')

params['k_1'] = 2*(12 * 2*E*I / H**3).subs(params).simplify().evalf(6)
params['k_2'] = 2*(12 * E*I / H**3).subs(params).simplify().evalf(6)
K = sp.Matrix([[k_1 + k_2, -k_2],[-k_2, k_2]])


render.eq_display(sp.MatrixSymbol('K', 2,2), K,
                  sp.MatrixSymbol('K', 2,2), K.subs(params),
                  )

    ⎡k₁ + k₂  -k₂⎤
K = ⎢            ⎥
    ⎣  -k₂    k₂ ⎦

    ⎡ 1.31836e+8⋅newton   -4.39453e+7⋅newton ⎤
    ⎢ ─────────────────   ───────────────────⎥
    ⎢       meter                meter       ⎥
K = ⎢                                        ⎥
    ⎢-4.39453e+7⋅newton    4.39453e+7⋅newton ⎥
    ⎢───────────────────   ───────────────── ⎥
    ⎣       meter                meter       ⎦

#### Eigenvektoren
##### Massenmatrix $\mathbf{M}$ 

Die Massenmatrix folgt dem gleichen Aufbau wie die Steifigkeitsmatrix. Es gelten die gleichen Vorzeichenregelungen.

In [15]:
m_1, m_2 = sp.symbols('m_1, m_2')
M = sp.Matrix([[m_1, 0],[0, m_2]])

render.eq_display(sp.MatrixSymbol('M', 2,2), M,
                  sp.MatrixSymbol('M', 2,2), M.subs(params))

    ⎡m₁  0 ⎤
M = ⎢      ⎥
    ⎣0   m₂⎦

    ⎡                   2                      ⎤
    ⎢40000⋅newton⋅second                       ⎥
    ⎢────────────────────           0          ⎥
    ⎢       meter                              ⎥
M = ⎢                                          ⎥
    ⎢                                         2⎥
    ⎢                      20000⋅newton⋅second ⎥
    ⎢         0            ────────────────────⎥
    ⎣                             meter        ⎦

##### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}$$

In [16]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

      33.1 
ω₁ = ──────
     second

      66.3 
ω₂ = ──────
     second

##### Eigenvektoren $\phi$

In [17]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')
params['phi_21'] = 1

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_11 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_11).values())[0]

params['phi_11'] = phi_11.subs(params).simplify() 




render.eq_display(sp.simplify((K-omega_1**2 *M)*phi_1),sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))

⎡                 ⎛                           ________________________________
⎢                 ⎜                          ╱   2   2                        
⎢             φ₁₁⋅⎝-k₂⋅m₁ + m₂⋅(k₁ + k₂) + ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅
⎢-k₂⋅m₂⋅φ₂₁ + ────────────────────────────────────────────────────────────────
⎢                                                                 2           
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                           m₂                
⎢                                                                             
⎢                 ⎛                          _________________________________
⎢                 ⎜                         ╱   2   2                         
⎢             φ₂₁⋅⎝k₂⋅m₁ - m₂⋅(k₁ + k₂) + ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅k
⎢-k₂⋅m₁⋅φ₁₁ + ────────────────────────────────────────────────────────────────
⎢                                                   

     ⎡0.5⎤
φ₁ = ⎢   ⎥
     ⎣1.0⎦

In [18]:
params['phi_22'] = 1

phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_12 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_12).values())[0]

params['phi_12'] = phi_12.subs(params).simplify() 




render.eq_display(sp.simplify((K-omega_2**2 *M)*phi_2),sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))

⎡                 ⎛                           ________________________________
⎢                 ⎜                          ╱   2   2                        
⎢             φ₁₂⋅⎝-k₂⋅m₁ + m₂⋅(k₁ + k₂) - ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅
⎢-k₂⋅m₂⋅φ₂₂ + ────────────────────────────────────────────────────────────────
⎢                                                                 2           
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                           m₂                
⎢                                                                             
⎢                 ⎛                          _________________________________
⎢                 ⎜                         ╱   2   2                         
⎢             φ₂₂⋅⎝k₂⋅m₁ - m₂⋅(k₁ + k₂) - ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅k
⎢-k₂⋅m₁⋅φ₁₂ + ────────────────────────────────────────────────────────────────
⎢                                                   

     ⎡-1.0⎤
φ₂ = ⎢    ⎥
     ⎣1.0 ⎦

##### Orthogonalitätsbedingung

Zur Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein.

Dies gilt es für die Massenmatrix zu kontrollieren:

$$\phi_1^T \mathbf{ M } \phi_1 \neq 0$$

$$\phi_2^T \mathbf{ M } \phi_2 \neq 0$$

$$\phi_2^T \mathbf{ M } \phi_1 = 0$$

Sowohl auch für die Steifigkeitsmatrix:

$$\phi_1^T \mathbf{ K } \phi_1 \neq 0$$

$$\phi_2^T \mathbf{ K } \phi_2 \neq 0$$

$$\phi_2^T \mathbf{ K } \phi_1 = 0$$

Angewendet auf die Aufgabe:


In [19]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_2.T*M*phi_1).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(3))

           ⎡                    2⎤
  T        ⎢3.0e+4⋅newton⋅second ⎥
φ₁ ⋅M⋅φ₁ = ⎢─────────────────────⎥
           ⎣        meter        ⎦

           ⎡                    2⎤
  T        ⎢6.0e+4⋅newton⋅second ⎥
φ₂ ⋅M⋅φ₂ = ⎢─────────────────────⎥
           ⎣        meter        ⎦

  T           
φ₂ ⋅M⋅φ₁ = [0]

  T           
φ₁ ⋅M⋅φ₂ = [0]

Für die Steifigkeitsmatrix:

In [20]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(4),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(4))

  T        ⎡3.3e+7⋅newton⎤
φ₁ ⋅K⋅φ₁ = ⎢─────────────⎥
           ⎣    meter    ⎦

  T        ⎡2.64e+8⋅newton⎤
φ₂ ⋅K⋅φ₂ = ⎢──────────────⎥
           ⎣    meter     ⎦

  T           
φ₂ ⋅K⋅φ₁ = [0]

  T           
φ₁ ⋅K⋅φ₂ = [0]

#### Eigenformen

![Die beiden Eigenformen skizziert](bilder/eigenformen_mms2.jpg){#fig-eigenformen}